## Accessing the NMR Metabolomics/Nightingale data

This notebook demonstrates how to extract all relevant NMR Metabolomics data from the database, and how to join them together to create a single dataset using R. Please note that this notebook is similar to A105_Export-participant-data_R.ipynb within this repository, but has been tailored specifically for the Metabolomics data.

Information on the Metabolomics data available can be found on our Showcase website:
- **[Category 220](https://ace.ndph.ox.ac.uk/ukb/label.cgi?id=220)** contains the data for each of the 251 metabolomic biomarkers (e.g. field 23474: 3-Hydroxybutyrate). Please also see the Resources tab within this page for more details on how the data was generated. The [companion document](https://biobank.ndph.ox.ac.uk/showcase/refer.cgi?id=2082) provides information on sample processing, ratio calculations, batch variation and reproducibility (the Phase 3 versions are the most recent).
- **[Category 221](https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=221)** includes any QC warnings associated with individual data points (e.g. field 23774: 3-Hydroxybutyrate, QC Flag)
- **[Category 222](https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=222)** contains other processing measures at a sample level (e.g. field 20282: Processing Batch).


All fields in these categories are labelled using [Instancing 2](https://biobank.ndph.ox.ac.uk/showcase/instance.cgi?id=2). This means that Instance_0 = the participants blood sample was collected from the initial assessment visit, Instance_1 = First repeat assessment visit etc. To get the dates that the blood sample was collected, you can join the data to [Field 53](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=53) (Date of attending Assessment Centre).

Note: If you are extracting all NMR Metabolomics data (with all QC metrics), the final dataset should contain ~500,000 rows (participants) and 790 columns.

##### Run info

- Runtime: <15 minutes
- Instance: mem1_hdd1_v2_x16 
- Cost: <£2.00

### This notebook depends on 
A **Spark Instance** - Make sure this option is selected when launching JupyterLab.

## 1. Install required packages
Function `p_load` from `pacman` loads packages into R. If the given package is missing p_load will automatically install it - this can take a considerable amount of time for a package that needs C or FORTRAN code compilation. The following packages are needed to run this notebook:

*Note*: If you wish to rerun this notebook, and avoid having to wait for the installation of the packages, creating a [snapshot](https://documentation.dnanexus.com/user/jupyter-notebooks#environment-snapshotshttps://documentation.dnanexus.com/user/jupyter-notebooks#environment-snapshots) of the environment is useful.

- `reticulate` - R-Python interface, required to use the `dxdata` package that connects to Spark database and allows retrieval of phenotypic data
- `Stringr` – Used for character manipulation 
- `Dplyr` – Tabular data manipulation in R
- `data.table` – Read data into data.table format 
- `arrow` - Input/output library for Apache binary files
- `readr` - Reading and writing tabular data in R


In [ ]:
 # Load required packages 
if(!require(pacman)) install.packages("pacman")
pacman::p_load(reticulate, stringr, dplyr, data.table, arrow, readr)

## 2. Import Python and Define your database
To improve the reproducibility of your notebooks and ensure they are portable across projects, it is better not to hardcode the connection to any database or dataset names. Instead, you can use the following code to automatically discover the database and dataset:

In [ ]:
# Import python packages
use_python("/opt/conda/bin/python") #pre-installed python environment that contains dxdata
dxdata <- import("dxdata")

In [ ]:
# Connect to the dataset
project_id <- Sys.getenv('DX_PROJECT_CONTEXT_ID') #find the project ID
record_id <- system("dx find data --type Dataset --delimiter ',' | awk -F ',' '{print $5}'" , intern = TRUE) #find the record ID
DATASET_ID <- paste0(project_id, ":", record_id)

# The DATASET_ID should be in the following format: 'project-XXXXXXXXXXX:record-YYYYYYYYYYYYYY' 
# Your project ID can also be found in the settings tab within your UKB-RAP project.
# To find the record ID, select the 'manage' tab within your UKB-RAP project, then select the '.dataset' file in the root directory.
# The record ID will appear under the 'ID' tab on the file info section on the right hand side of the screen.

print(paste0("DATASET_ID: ", DATASET_ID))
# Note: if you have more than one dataset ID retrieved, you can use the cell below this one to automatically find the most recent one in the project.

In [ ]:
# If you have more than one dataset ID available in your project, use the most recent one:
# This function extracts the record ID and its created date:
get_record_info <- function(dataset_id) {
  out <- system(paste("dx describe", dataset_id), intern = TRUE) # dx describe finds details about the dataset_ids
  id_line <- out[grep("^ *ID", out)][1] # extract the the ID line
  record_id <- sub("^ *ID[[:space:]]+", "", id_line) # get the record ID
  
  created_line <- out[grep("^ *Created[[:space:]]", out)] #this is the created date/time
  created_line <- created_line[!grepl("Created by", created_line)][1] #ignore the "created by" information
  created <- sub("^ *Created[[:space:]]+", "", created_line)
  created <- as.POSIXct(created, format = "%a %b %d %H:%M:%S %Y", tz = "UTC") # Convert date to POSIXct
  
  data.frame(record_id = record_id, created = created, stringsAsFactors = FALSE) # Return as a data frame
}

# Apply the function to all dataset IDs and combine the results into one data frame
record_id_info <- do.call(rbind, lapply(DATASET_ID, get_record_info))

record_id_info

In [ ]:
record_id <- record_id_info %>%
  filter(created == max(created, na.rm = TRUE)) %>%
  pull(record_id)

record_id

In [ ]:
#re-run the dataset id with the selected record ID
DATASET_ID <- paste0(project_id, ":", record_id)

In [ ]:
dataset <- dxdata$load_dataset(id=DATASET_ID) #load the metadata for the dataset

In [ ]:
# Identify the database and app ID
database_path <- system("dx find data --class database", intern =TRUE) #find the path to the database
app_substring <- na.omit(str_extract(database_path, '(app\\d+_\\d+)')) # extract the app ID from the database path
database_substring <- str_extract(database_path[str_detect(database_path, app_substring)], 'database-([A-Za-z0-9]+)') %>% tolower()  %>% str_replace("database-", "database_") #extract the database name
database <- paste0(database_substring, "__", app_substring) #generate the final database identifier

# The database identifier should be in the following format: database_ZZZZZZZZZZ__appQQQQQQ_NNNNNNNNN'
# To manually find your database path, go to the 'manage' tab within your UKB_RAP project, then select the '.database' file in the root directory.
# The first part of the database identifier is the file ID (found on the right hand side of the screen). The second part of the identifier is the file name

print(paste0("database = ", database))

## 3. Extract the data from the database

The NMR metabolomics data is stored within the main participant data. The participant data is split into multiple tables, which can make it difficult to query them via SQL. Functions from `dxdata` via pyspark allow access to the participant data.

In [ ]:
# Connect to Pyspark via dxdata
engine <- dxdata$connect(dialect="hive+pyspark")

In [ ]:
# Load in the data from your project
dataset = dxdata$load_dataset(id=DATASET_ID)
# Select the participant table (this is where the metabolomics data is stored)
participant_database = dataset["participant"]

##### Get the Corresponding Field Schema Data
Importing the schema data from the 'Showcase metadata' folder in the project allows you to explore the list of fields and map field ids to names, making it easier to search within the data.

Within the database, fields are identified by:
* **Field id**: this correspond to the unique field integer identifier (e.g. 94)
* **Field title**: this is the title of the field  (e.g. Diastolic blood pressure, manual reading)
* **Field name**: this includes the entity, field id, field instance and array (e.g. 'p94_i0_a0', 'p94_i0_a1', 'p94_i1_a0', 'p94_i1_a1', 'p94_i2_a0', 'p94_i2_a1', 'p94_i3_a0', 'p94_i3_a1')

In [ ]:
# Load in schema data (this contains searchable field names)
field_schema <- fread("/mnt/project/Showcase metadata/field.tsv")

# Download the data dictionary schema into the environment (this contains field ids)
system(paste0("dx extract_dataset ", DATASET_ID, " -ddd"), intern = TRUE)

# Read in the data dictionary
datadict <- data.table::fread(list.files(pattern = "data_dictionary\\.csv$", full.names = TRUE))


print(paste0("The data dictionary contains ", nrow(datadict), " rows."))

If you prefer to extract all metabolomics data run the following cell:

In [ ]:
# Filter the field_schema for all metabolomics data
field_ids_of_interest<- field_schema %>% 
                         filter(main_category %in% c(220,221,222)) %>% # These are all the categories associated with the metabolomics data
                         pull(field_id)

#If you are extracting all metabolomics fields, you should have 512 fields at this point.
print(paste0(length(field_ids_of_interest), " fields of interest have been identified."))

Alternatively, you can select specific fields of interest:

In [ ]:
# Filter the field_schema for a specific subset of the metabolomics data
field_ids_of_interest<- field_schema %>% 
                         filter(main_category == 222| # This category includes all the processing indicators (useful for QC)
                            title %in% c("Acetate", "Acetate, QC Flag", "Date of attending Assessment Centre")| #Specific metabolite and its related QC field (names as they appear on showcase)
                            field_id==53) %>% #Date of attending assessment centre (used to identify when the blood sample was collected from the participant)  
                         pull(field_id)

In [ ]:
# Often Field ids have multiple instances - this function will find all the associated field ids
fields_for_id <- function(field) {
    regex <- paste0('^p', field, '(?![0-9])')
    fields <- dplyr::filter(datadict, stringr::str_detect(name, regex)) %>%
        dplyr::pull(name)
    return(fields)
}

In [ ]:
# Get all data from the database for all fields of interest using the schema data (using the function above)
# 'eid' is added manually because it is not included within 'field_schema'
complete_field_ids <- c('eid', unlist(lapply(field_ids_of_interest, fields_for_id)))

#if you are extracting the full metabolomics data, you should have 790 fields in this list
length(complete_field_ids)

In [ ]:
# Convert field names into dxdata objects
# 'iterate()' is needed because the 'find_fields()' output is an itorator object 
# The resulting object is a list of `Field` objects (object specific to dxdata).
# There will be more than the original 512 field ids here as this includes each field split up by instance and array
complete_field_ids <- iterate(participant_database$find_fields(names=complete_field_ids))

##### Assign column names

In [ ]:
# Clean the column names for PySpark retrieval
# PySpark columns must have no spaces or punctuation - create a function for this
complete_col_names_clean <- lapply(seq_along(complete_field_ids), function(i) {
    clean_title <- gsub(" ", "_", complete_field_ids[[i]]$title) # replace spaces with underscores
    clean_title <- gsub("[^a-zA-Z0-9_]", "", clean_title) # remove special characters
    setNames(list(clean_title), complete_field_ids[[i]]$column_name)

})

In [ ]:
complete_col_names_clean <- do.call(c, complete_col_names_clean) #run the function above
complete_col_names_clean <- dict(complete_col_names_clean) # Convert to a python dictionary

#There should be 790 fields here
length(complete_col_names_clean)

In [ ]:
#create the look up table between the original column names and the clean versions
mapping_df <- data.frame(
    column_name = sapply(complete_field_ids, function(x) x$column_name),
    clean_title = sapply(complete_field_ids, function(x) {
        t <- gsub(" ", "_", x$title)
        gsub("[^A-Za-z0-9_]", "", t) }), stringsAsFactors = FALSE)

In [ ]:
# Write the complete list of field ids to csv (this is needed when the kernel is restarted below)
write.csv(mapping_df, "field_name_mapping.csv", row.names = FALSE)

In [ ]:
# Retrieve the data using PySpark - This is the main extraction step, which returns and pyspark df
data_of_interest <- participant_database$retrieve_fields(engine=engine, fields=complete_field_ids, column_aliases = complete_col_names_clean)

# Note: If an error occurs here, double check that the instance was started with a spark cluster, and that the database path is correct.
# In the 'monitor' tab of your project, the Executable column for the instance should contain 'JupyterLab with Spark Cluster'.

## 4. Save the data

In [ ]:
# Save the PySpark data frame as a parquet file
system('hadoop fs -rm -r -f data_of_interest.parquet', intern = TRUE) #remove any previous data saved
data_of_interest$write$parquet('data_of_interest.parquet') #write using PySpark

In [ ]:
# Copy the parquet file into local env
if(dir.exists('data_of_interest.parquet')) unlink("data_of_interest.parquet", recursive=TRUE)
system('hadoop dfs -copyToLocal data_of_interest.parquet', intern = TRUE)

# This file can then be uploaded to your project, which can be useful particularly for large files:
# system("dx upload -r data_of_interest.parquet") 
# To then use this file at a later date, you can use open_dataset() and collect() within RStudio for instance.

## 5. Reading in the Parquet File

In [ ]:
# Important: Restart the kernel at this point to disconnect from Pyspark. 
# After restarting the kernel, you will need to reconnect to pyspark to extract any more data (it may be safest to re-run the notebook from the beginning again if so)

# Re-load packages 
if(!require(pacman)) install.packages("pacman")
pacman::p_load(reticulate, stringr,  dplyr, data.table, arrow, readr)

In [ ]:
# Load in the parquet file
data_of_interest_ds <- arrow::open_dataset('data_of_interest.parquet')

#Bring the data in as an R data frame
data_of_interest_tbl <- data_of_interest_ds %>% collect()

## 6. Optional Data Manipulation

#### Remove any measurements associsted with a QC_Flag

In [ ]:
#Generate a table that maps the metabolite data to the coresponding QC Flag column
QC_flag_mapping <- tibble(col = names(data_of_interest_tbl)) %>%
  filter(
    str_detect(col, "_Instance_\\d+$"), #find any column that has 'instance' in its name
    !str_detect(col, "_QC_Flag_") #exclude any QC_Flag columns
  ) %>%
  mutate(
    instance = str_extract(col, "__Instance_\\d+$"), #find out what instance its from
    base = str_remove(col, "__Instance_\\d+$"), #find the instance suffix
    qc_col = paste0(base, "_QC_Flag", instance), #create a column for the expected QC_flag column
    qc_exists = qc_col %in% names(data_of_interest_tbl) #see if that QC column exists in the data
  ) %>% 
filter(qc_exists==TRUE) %>% #keep the columns that have a corresponding QC_Flag column
  select(value_col = col, qc_col)

In [ ]:
# Check this here:
QC_flag_mapping

In [ ]:
data_clean <- data_of_interest_tbl

for (i in seq_len(nrow(QC_flag_mapping))) { #loop over the QC_flag_mapping table
  value <- QC_flag_mapping$value_col[i] #set the value column
  qc_flag  <- QC_flag_mapping$qc_col[i] #set the QC_Flag column

  data_clean[[value]] <-
    replace(
      data_clean[[value]], #take the value column
      !is.na(data_clean[[qc_flag]]), #if the QC flag is not NA (i.e. there was a QC flag for that data point)
      NA #replace the value with NA in the corresponding column
    )
}

#### Convert the field titles to field names

In [ ]:
#load in file generated that links the original file name to the clean version
field_name_mapping <- read_csv("field_name_mapping.csv", show_col_types = FALSE)
# Replace the column names 
new_names <- field_name_mapping$column_name[ match(names(data_clean), field_name_mapping$clean_title) ] 
names(data_clean) <- new_names

#### Write the data as CSV

In [ ]:
write.csv(data_clean, 'data_clean.csv')

In [ ]:
system("dx upload data_clean.csv")